In this lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- answering queries using this index,
- and organizing it as a simple web server.

Second part is devoted to spellchecking.

# 1. [45] Building inverted index and answering queries

## 1.1. [5] Preprocessing

First, we need a unified approach to documents preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library or any other you know.


In [2]:
import nltk
nltk.download('punkt')

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        return nltk.word_tokenize(text)

    
    def stem(self, word, stemmer):
        return stemmer.stem(word)

    
    def is_apt_word(self, word):
        if word in self.stop_words or not word.isalpha() or len(word) == 1:
            return False
        return True

    
    def preprocess(self, text):
        preprocessed = self.tokenize(text.lower())
        preprocessed = [i for i in preprocessed if self.is_apt_word(i)]
        preprocessed = [self.stem(i, self.ps) for i in preprocessed if self.is_apt_word(i)]
        return preprocessed

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\demo8\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.1.1. Tests

In [3]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

test = "I am writing super test test1, ha h a !"
print(prep.preprocess(test))

['am', 'write', 'super', 'test', 'ha']


## 1.2. [25] Crawling and Indexing

### 1.2.1. [5] Base classes

Here are some base classes we will need for writing our indexer. The code from the first lab's solution is given, but note that you will need to change some of it, namely, the `parse` function (it is also possible to use your own implementation from the first homework, but make sure that it is correct). The reason is it always makes complete parsing, which we want to avoid when we only need links, for example, or a specific portion of text.

In [5]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os


class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                self.content = ""
                return False
        except:
            self.content = ""
            return False

    def persist(self, path):
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)

# This class returns anchors only (content and filtering to articles will be latter)
class HtmlDocument(Document):
    def __init__(self, url):
        super().__init__(url)
        super().download()

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self):
        
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
            
        
        model = BeautifulSoup(self.content)
        
        self.anchors = []
        a = model.find_all('a')
        for anchor in a:
            href = self.normalize(anchor.get('href'))
            self.anchors.append(href)

In [6]:
doc = HtmlDocument("https://www.reuters.com/technology")
doc.download()
doc.parse()
print(doc.anchors)

['https://www.reutersagency.com/en/media-center/welcome-to-the-new-reuters-com/', 'https://www.reuters.com/technology#main-content', 'https://www.reuters.com/', 'https://www.reuters.com/world/', 'https://www.reuters.com/business/', 'https://www.reuters.com/markets/', 'https://www.reuters.com/breakingviews/', 'https://www.reuters.com/signin/', 'https://www.reuters.com/technology/microsoft-says-group-behind-solarwinds-hack-now-targetting-government-agencies-2021-05-28/', 'https://www.reuters.com/technology/colonial-pipeline-says-experiencing-network-issues-2021-05-28/', 'https://www.reuters.com/technology/apple-delay-launch-podcast-subscription-service-until-june-2021-05-28/', 'https://www.reuters.com/technology/us-sec-charges-five-individuals-involved-crypto-lending-program-2021-05-28/', 'https://www.reuters.com/business/call-amazon-consider-blue-collar-director-wins-17-support-2021-05-28/', 'https://www.reuters.com/business/finance/bitcoin-slumps-8-it-heads-bruising-monthly-drop-2021-0

### 1.2.2. [15] Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (of a form https://www.reuters.com/...) for visiting. To speed up, doesn't consider for visiting pages with content type other than html: '.pdf', '.mp3', '.avi', '.mp4', '.txt'. If encounters an article page (of a form https://www.reuters.com/article/...), saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`, `doc_id:url`
    - `index`, `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    - `doc_lengths`, `doc_id:doc_length` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.


**Extra task \* (no penalty to skip)** In real industrial systems a crawler would pass the links to the dedicated service that would load their contents in a bunch of parallel threads. Implement such a service - get urls as inputs, load page contents in parallel and return filenames on disk, which are then processed by indexer.

In [10]:
# Support class for getting urls of articles only from https://www.reuters.com/
class ArticleCrawler:
    def __init__(self, limit=None):
      # Collecting visited url
      self.visited = []
      # Set is cooler than Queue, because it automatically checks duplicates
      self.articles = set()
      # Max of articles
      self.limit = limit or 100000000000000

    def check(self, url):
        # Correct URL & Didn't see before & from https://www.reuters.com/... & just html
        return  (url is not None) and \
                (url not in self.visited) and \
                (self.root in url) and \
                (url[-4:] not in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'))

    # Find all refs and add them to the set
    def crawling(self, url, queue):
        if url in self.visited:
            return
        self.visited.append(url)
        doc = HtmlDocument(url)
        doc.parse()
        urls = doc.anchors
        # Add only correct refs
        for i in urls:
            if self.check(i):
                # refs from queue would be checked latter
                queue.add(i)
                # If this is an article, add  also to the set of articles
                if "/article/" in i:
                    self.articles.add(i)

    def get_urls(self, source, depth = 1):
        # https://www.reuters.com
        self.root = source.split(".com/")[0]
        # Refs from source
        old_set = set()
        self.crawling(source, old_set)
        # BFS
        for i in range(depth - 1):
            # New level of refs tree
            new_set = set()
            for url in old_set:
                self.crawling(url, new_set)
                # If self.crawling found enough articles
                if len(self.articles) >= self.limit:
                    return list(self.articles)[:self.limit]
            old_set = new_set
            print("Depth", i, " is done. Found ", len(self.articles))
        # If limit=None (or greater than count of articles), return the all findings
        return list(self.articles)

In [11]:
from collections import Counter
import pickle
import os

class Indexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()

    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        # Articles urls
        urls = ArticleCrawler(limit).get_urls(source, depth)
        # Get the content from articles
        for i in range(len(urls)):
            self.doc_urls[i] = urls[i]
            # Head & body of html
            content = BeautifulSoup(str(requests.get(urls[i], allow_redirects=False).content))
            head = content.findAll('h1')
            doc = content.findAll('article')
            art_text = ""
            for j in doc:
                art_text += j.text
            for j in head:
                art_text += " " + j.text
            self.index_doc(art_text, i)
        with open(os.path.join(collection_path, 'doc_urls.p'), 'wb') as fp:
            pickle.dump(self.doc_urls, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(collection_path, 'inverted_index.p'), 'wb') as fp:
            pickle.dump(self.index, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(collection_path, 'doc_lengths.p'), 'wb') as fp:
            pickle.dump(self.doc_lengths, fp, protocol=pickle.HIGHEST_PROTOCOL)
        # Difference from template: list of urls instead of HtmlDocument,
        # because it is parsed in another place
        return self.doc_urls.values()

    def index_doc(self, doc, doc_id):
        preproc = self.prep.preprocess(doc)
        count = Counter()
        for word in preproc:
            count[word] += 1
        for word in count:
            if len(self.index.setdefault(word, [])) == 0:
                self.index.setdefault(word, []).append(count[word])
            else:
                self.index.setdefault(word, [])[0] +=count[word]
            self.index.setdefault(word, []).append((doc_id, count[word]))
        self.doc_lengths[doc_id] = len(preproc)

### 1.2.3. Tests

In [12]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/technology", 2, "test_collection", 5):
    print(k, c)
    k+=1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

1 https://www.reuters.com/article/factcheck-nasa-zodiac/fact-check-false-posts-about-nasa-changing-the-zodiac-resurface-idUSL2N2NF2AW
2 https://www.reuters.com/article/factcheck-deer-man/fact-check-story-of-man-who-self-identifies-as-a-deer-stems-from-satire-idUSL2N2NE2BN
3 https://www.reuters.com/article/factcheck-microchip-vaccine/fact-check-pictured-microchip-is-unrelated-to-covid-19-vaccine-idUSL2N2NF0XQ
4 https://www.reuters.com/article/factcheck-chelsea-clinton/fact-check-fabricated-chelsea-clinton-tweet-about-racial-rehab-idUSL2N2NE2LR
5 https://www.reuters.com/article/factcheck-trojan-preowned/fact-check-trojan-is-not-selling-pre-owned-condoms-idUSL2N2NF296


### 1.2.4. Building an index

In [13]:
# Take some time...
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/", 3, "docs_collection"):
    print(k, c)
    k+=1

Depth 0  is done. Found  51
Depth 1  is done. Found  856
1 https://www.reuters.com/article/factcheck-maskless-dems/fact-check-video-of-maskless-democrats-taken-after-cdcs-latest-guidance-idUSL2N2ND1RD
2 https://www.reuters.com/article/us-usa-biden-budget/money-is-cheap-lets-spend-it-white-house-6-trillion-budget-message-idUSKCN2D9105#main-content
3 https://www.reuters.com/article/jd-logistics-listing/update-4-jd-logistics-shares-lose-steam-to-close-3-3-higher-idUSL2N2NF02L
4 https://www.reuters.com/article/us-china-usa-trade/china-u-s-can-find-common-ground-on-tariff-exclusions-chinese-think-tank-says-idUSKCN2DA06J
5 https://www.reuters.com/article/us-weir-group-divestiture-caterpillar/weir-group-to-sell-oil-gas-division-to-caterpillar-shares-surge-idUSKBN26Q0O6
6 https://www.reuters.com/article/us-usa-military-warren/sen-warren-seeks-investigation-of-landlord-at-u-s-air-force-base-idUSKCN1TS1YY
7 https://www.reuters.com/article/rivian-ipo/update-1-electric-vehicle-firm-rivian-could-se

### 1.2.5. [5] Index statistics

Load an index and print the statistics.

In [14]:
# load index, doc_lengths and doc_urls
with open(os.path.join('docs_collection', 'inverted_index.p'), 'rb') as fp:
    index = pickle.load(fp)
with open(os.path.join('docs_collection', 'doc_lengths.p'), 'rb') as fp:
    doc_lengths = pickle.load(fp)
with open(os.path.join('docs_collection', 'doc_urls.p'), 'rb') as fp:
    doc_urls = pickle.load(fp)
    
    
print('Total index length', len(index))
print('\nTop terms by number of documents they apperared in:')
sorted_by_n_docs = sorted(index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])
print('\nTop terms by overall frequency:')
sorted_by_freq = sorted(index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index length 9917

Top terms by number of documents they apperared in:
[('min', 478), ('reuter', 477), ('trust', 472), ('standard', 472), ('thomson', 471), ('principl', 471), ('said', 401), ('edit', 399), ('have', 304), ('which', 298), ('not', 286), ('report', 281), ('more', 263), ('thi', 260), ('after', 257), ('also', 251), ('year', 241), ('compani', 231), ('but', 222), ('would', 220)]

Top terms by overall frequency:
[('said', 1857), ('reuter', 1743), ('have', 970), ('not', 883), ('report', 726), ('more', 614), ('which', 603), ('compani', 596), ('after', 581), ('they', 568), ('had', 559), ('year', 555), ('thi', 543), ('their', 529), ('been', 518), ('would', 514), ('or', 512), ('standard', 505), ('but', 505), ('trust', 492)]


## 1.3. [15] Answering query

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [15]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        count = Counter()
        for word in query:
            count[word] += 1
        return count
    
    @staticmethod
    def boolean_retrieval(query, index):
        retrieval = []
        # AND boolean retrieval
        for word in query:
            # [(doc_id, term_freq)]
            freqs = index[word][1:]
            if not retrieval:
                # i[0] - document id
                retrieval = [i[0] for i in freqs]
            else:
                # Intersection
                retrieval = [i[0] for i in freqs if i[0] in retrieval]
        return set(retrieval)

    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        okapi = {}
        avg_len = 0
        N = len(doc_lengths)
        for i in doc_lengths.values():
            avg_len += i
        avg_len /= N
        for word in query:
            inv_ind = index[word]
            n_q = len(inv_ind[1:])
            IDF = math.log((N - n_q + 0.5) / (n_q + 0.5))
            for freq in inv_ind[1:]:
                okapi.setdefault(freq[0], 0)
                okapi[freq[0]] += IDF * freq[1] * (k1 + 1) \
                                / (freq[1] + k1 * (1 - b + b * doc_lengths[freq[0]] / avg_len))
        return okapi

### 1.3.1. Tests

In [16]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'xx': [2, (1, 1), (2, 1)], 'yy': [2, (1, 1), (3, 1)], 'zz': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('xx zz')
test_query2 = QueryProcessing.prepare_query('xx yy')

assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]


In [ ]:
#TODO write a web-service that answers queries using inverted index

# 2. [55] Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. User knows that he doesn't know the correct spelling OR he wants to get the results that follow some known pattern, so he uses so called wildcards - queries like `retr*val`;
2. User doesn't know the correct spelling OR he doesn't care OR he's in a rush OR he expects his mistakes will be corrected OR your option, so he makes mistakes and we need to handle them using:

    2.1. Simple spellchecker by Peter Norvig;
    
    2.2. Phonetic correction by means of Soundex algorithm;
    
    2.3. Trigrams with Jaccard coefficient.

## 2.1. [20] Handling wildcards

We will handle wildcard queries using k-grams. K-grams is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf) *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 2.1.1. Downloading the dataset

In [23]:
import urllib.request
data_url = "https://raw.githubusercontent.com/hsu-ai-course/hsu.ai/master/code/datasets/nlp/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
'''
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))'''
# Correct facts (21-24 facts repeat)
with open("facts.txt", 'r') as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


### 2.1.2. [20] Implementation of search

In [24]:
import re
from nltk import word_tokenize
from nltk.util import ngrams

def build_inverted_index_orig_forms(documents):
    inverted_index = {}
    for raw_doc in documents:
        # Split id of fact and its content
        doc = raw_doc.split(".")
        doc_id = int(doc[0])
        # Content
        doc_proc = word_tokenize(" ".join(doc[1:]).lower())
        # Copypaste from 1.2.2
        count = Counter()
        for word in doc_proc:
            count[word] += 1
        for word in count:
            if len(inverted_index.setdefault(word, [])) == 0:
                inverted_index.setdefault(word, []).append(count[word])
            else:
                inverted_index.setdefault(word, [])[0] +=count[word]
            inverted_index.setdefault(word, []).append((doc_id, count[word]))
    return inverted_index



def build_k_gram_index(inverted_index, k):
    k_gram_index = {}
    # For each word in vocab create a list on k-grams
    for word in inverted_index.keys():
        k_grams = ngrams("$" + word + "$", k)
        # Add 'word' to dict with key 'k_gram'
        for i in k_grams:
            k_gram = "".join(i)
            k_gram_index.setdefault(k_gram, []).append(word)
    return k_gram_index

# Divide wildcard by k-grams; for each k-gram return its words;
# make an intersection of words
def generate_wildcard_options(wildcard, k_gram_index, inverted_index, k=3):
    wildcard = "$" + wildcard + "$"
    patterns = wildcard.split("*")
    words = []
    # ['re', 'ed']
    for gram in patterns:
        K = len(gram)
        # Create another k_gram_index, if k in k_gram_index is not K
        if K != k:
            k_gram_index_ = build_k_gram_index(inverted_index, K)
        else:
            k_gram_index_ = k_gram_index
        voc_words = k_gram_index_[gram]
        if not words:
            words = voc_words
        else:
            # Intersection between words and the new list
            words = [i for i in voc_words if i in words]
    return words

# Get matching of wildcard; for each matching find documents where it occurs
def search_wildcard(wildcard, k_gram_index, index, docs):
    words = generate_wildcard_options(wildcard, k_gram_index, index)
    ids = []
    for word in words:
        docs_list = index[word][1:]
        for i in docs_list:
            ids.append(i[0])
    # docs start from 1
    return [docs[id-1] for id in ids]

### 2.1.3. Tests

In [41]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)
print(k_gram_index)
wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

{'$if': ['if'], 'if$': ['if'], '$yo': ['you', 'your', 'york', 'yourself'], 'you': ['you', 'your', 'yourself'], 'ou$': ['you'], '$so': ['somehow', 'southern', 'son', 'soda', 'soft-boiled', 'so', 'someone', 'sometimes', 'solar', 'some'], 'som': ['somehow', 'someone', 'sometimes', 'some'], 'ome': ['somehow', 'women', 'someone', 'sometimes', 'comes', 'homeless', 'become', 'some'], 'meh': ['somehow'], 'eho': ['somehow'], 'how': ['somehow', 'however'], 'ow$': ['somehow', 'snow', 'cow', 'follow', 'now', 'low'], '$fo': ['found', 'food', 'forelegs', 'for', 'foot', 'folded', 'fountain', 'follow', 'fourth', 'footprints', 'founders', 'four'], 'fou': ['found', 'fountain', 'fourth', 'founders', 'four'], 'oun': ['found', 'pounds', 'around', 'fountain', 'pronounce', 'pound', 'bloodhound', 'mountain', 'bounce', 'founders', 'amount', 'country'], 'und': ['found', 'under', 'pounds', 'around', 'underneath', 'pound', 'bloodhound', 'understanding', 'founders', 'sunday'], 'nd$': ['found', 'land', 'and', 'arou

## 2.2. [35] Handling typos

### 2.2.1 Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [ ]:
!pip install jsonlines

In [26]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [27]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

### 2.2.2. [5] Build a dataset vocabulary
We will need it for Norvig's spellchecker as well as for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [28]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [29]:
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

63724

In [30]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6193), ('de', 82), ('deutsch', 4), ('nocomments', 2), ('you', 42075), ('can', 26027), ('disable', 532), ('comments', 360), ('for', 44756), ('the', 207017)]


### 2.2.3. [25] Implement context-independent spellcheker ##

0) Write code to compute editorial distance

1) [Norvig's corrector](https://norvig.com/spell-correct.html)

2) [Soundex](https://en.wikipedia.org/wiki/Soundex)

3) Trigrams with Jaccard coefficient.

#### 2.2.3.0. [5] Editorial distance

Frequently used distance measure between two character sequences. We will use this distance to sort Soundex search results.

In [31]:
def edit_dist(s1, s2) -> int:
    # Stolen from Wiki
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition

    return d[lenstr1-1,lenstr2-1]

In [32]:
# tests

assert edit_dist("korrectud", "corrected") == 2, "Edit distance is computed incorrectly"
assert edit_dist("soem", "some") == 1, "Edit distance is computed incorrectly"
assert edit_dist("one", "one") == 0, "Edit distance is computed incorrectly"

#### 2.2.3.1. [5] Norvig's spellchecker

In [33]:
# Stolen from Peter Norvig
def P(word, N=sum(vocabulary.values())):
    "Probability of `word`."
    return vocabulary[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in vocabulary)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def fix_typo_norvig(word) -> str:
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

In [34]:
# tests

assert fix_typo_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_norvig("speling") == "spelling", "Norvig's correcter doesn't work"

assert fix_typo_norvig("wort") == "work", "Norvig's correcter doesn't work"
assert fix_typo_norvig("mornin") == "morning", "Norvig's correcter doesn't work"
assert fix_typo_norvig("cofee") == "coffee", "Norvig's correcter doesn't work"

#### 2.2.3.2. [10] Soundex 

For cases when the exact spelling is unknown, phonetic algorithms such as Soundex can be very helpful - they allow user to type a word the way he thinks it should sound, and then suggest the corrrect version. Go through *chapter 3.4* to understand how Soundex algorithm works.

In [35]:
# letter -> number from soundex
def code_map(letter):
    if letter in ['b', 'f', 'p', 'v']:
        return '1'
    elif letter in ['c', 'g', 'j', 'k', 'q', 's', 'x', 'z']:
        return '2'
    elif letter in ['d', 't']:
        return '3'
    elif letter in ['l']:
        return '4'
    elif letter in ['m', 'n']:
        return '5'
    elif letter in ['r']:
        return '6'
    else:
        return '0'

def produce_soundex_code(word):
    code = ""
    for letter in word:
        code += code_map(letter)
    # Flag, if catch a pair
    pair = False
    hash_word = code[0]
    # Remove pairs
    for i in range(1, len(code)):
        # If there is a pair, drop the second letter
        if code[i] == code[i-1] and not pair:
            pair = True
        else:
            hash_word += code[i]
            pair = False
    # Replace the code of first letter by letter itself
    hash_word = word[0] + hash_word[1:]
    # Remove zeroes
    hash_word = hash_word.replace('0', '')
    # Add trailing zeroes, if needed
    while len(hash_word) < 4:
        hash_word += '0'
    return hash_word[:4]


def build_soundex_index(dictionary):
    soundex_index = {}
    for word in dictionary.keys():
        soundex_index.setdefault(produce_soundex_code(word), []).append(word)
    return soundex_index


def fix_typo_soundex(word, soundex_index) -> list:
    soundex_index.setdefault(produce_soundex_code(word), [''])
    # Sorting by distance between possible correction and word
    return sorted(soundex_index[produce_soundex_code(word)], key=lambda x: edit_dist(x, word))

In [36]:
# tests

soundex_index = build_soundex_index(vocabulary)
print(soundex_index)
code1 = produce_soundex_code("britney")
code2 = produce_soundex_code("breatany")
print(code1, code2)
assert code1 == code2
print(fix_typo_soundex("enouhg", soundex_index))
assert "enough" in fix_typo_soundex("enouhg", soundex_index), "Assert soundex failed"

{'f523': ['function', 'functions', 'functional', 'functor', 'functionality', 'fenced', 'finished', 'functionally', 'functionalities', 'fencedfeedbackpanel', 'functionpointerwithcontext', 'functioninfo', 'functionname', 'fnstring', 'functorial', 'functionnode', 'functionnality', 'functioning', 'finestra', 'functionalplus', 'funcdef', 'functionexpression', 'functiondeclaration', 'functionstmt', 'funcction', 'funktion', 'functionscript', 'functionalsourceset', 'functionevent', 'funciton', 'funciotn', 'functionalinterfaces', 'finishdynamicimport', 'functionreceivedarguments', 'functionalizing', 'functors', 'functionpass', 'functiontype', 'functons', 'funxtion', 'functionnal', 'functionbasedbushingforce', 'finest', 'functionfactory', 'functiondefinitionast', 'finishwitherror', 'functools', 'functionalty', 'functionnalities', 'functionalcollectiondata', 'functionaltabledata', 'functionalize'], 'd000': ['de', 'due', 'do', 'day', 'dd', 'd', 'dw', 'dt', 'dy', 'di', 'da', 'die', 'du', 'dao', 'da

#### 2.2.3.3. [5] Trigrams with Jaccard coefficient

In [37]:
def IoU(A, B):
    return len([i for i in A if i in B]) / len(list(set(A) | set(B)))

def fix_typo_kgram(word, k_gram_index, k =3) -> list:
    k_grams = ngrams("$" + word + "$", k)
    word_sets = []
    for i in k_grams:
        # If there is no grams (to avoid throws)
        k_gram_index.setdefault(''.join(i), [''])
        word_sets.append(k_gram_index[''.join(i)])
    matches = []
    for i in range(len(word_sets)):
        for j in range(i+1, len(word_sets)):
            if IoU(word_sets[i], word_sets[j]) > 0.0:
                matches += [i for i in word_sets[i] if i in word_sets[j]]
    matches = set(matches)
    return sorted(matches, key=lambda x: edit_dist(x, word)) or ['']

In [38]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'eno', 'enought', 'enoent', 'enosys', 'enomem', 'enospc', 'renounce', 'enormous', 'exogenous', 'enormously', 'endogenous', 'homogenous', 'hetrogenous', 'heterogenous', 'enodeuricomponent', 'ensuretablenotexists', 'testnotenoughqueryargs', 'entitytypenotfoundexception']


## 2.2.4. [5] Estimate quality

In [40]:
norvig, soundex, kgram = 0, 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, ss, sk = src.lower(), src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, ws, wk = fix_typo_norvig(word), \
                         fix_typo_soundex(word, soundex_index)[0], \
                         fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            ss = ss.replace(word, ws)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    soundex += int(ss == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"Soundex accuracy ({soundex}) = {soundex / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")


Norvig accuracy (2430) = 0.243
Soundex accuracy (1845) = 0.1845
k-gram accuracy (1952) = 0.1952
